# Audio Playground

## Setup

In [1]:
!nvidia-smi

Fri Dec  9 13:38:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   57C    P0   237W / 300W |  14674MiB / 32510MiB |     89%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:62:00.0 Off |                    0 |
| N/A   

In [5]:
import IPython.display as ipd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import numpy as np
import os, sys
import torch
from torch.utils.data import DataLoader
import librosa
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
PROJECT_ROOT = "/project/fdreyer/projects/vqvae-vc"
EMONYMOUS_DATA_DIR = "/data/share/emonymous"
VCTK_DATA_DIR = os.path.join(EMONYMOUS_DATA_DIR, "vctk")
sys.path.append(PROJECT_ROOT)
device = "cuda" if torch.cuda.is_available() else "cpu"
from src.data.datasets import VCTKDataset
from src.data.utils import vc_pad_collate

## Load Audio File with Librosa

In [6]:
path_to_audio = os.path.join(VCTK_DATA_DIR, "wav48_silence_trimmed", "s5", "s5_301_mic2.flac")
audio, sr = librosa.load(path_to_audio, sr=None)

In [161]:
ipd.Audio(audio, rate=sr)

## Chop audio by silence

In [162]:
def pydub_to_np(audio: pydub.AudioSegment) -> (np.ndarray, int):
    """
    Converts pydub audio segment into np.float32 of shape [duration_in_seconds*sample_rate, channels],
    where each value is in range [-1.0, 1.0].
    Returns tuple (audio_np_array, sample_rate).
    """
    return np.array(audio.get_array_of_samples(), dtype=np.float32).reshape((-1, audio.channels)) / (
            1 << (8 * audio.sample_width - 1)), audio.frame_rate

NameError: name 'pydub' is not defined

In [163]:
sound = AudioSegment.from_file(path_to_audio, format="flac")

In [164]:
chunks = split_on_silence(sound, min_silence_len=100, silence_thresh=sound.dBFS-16, keep_silence=100)

In [165]:
chunks = [pydub_to_np(chunk) for chunk in chunks]

NameError: name 'pydub_to_np' is not defined

In [166]:
for a, sr in chunks:
    ipd.display(ipd.Audio(np.squeeze(a), rate=sr))

ValueError: too many values to unpack (expected 2)

## Check VCTKDataset and vc_pad_collate Functionality

In [10]:
speaker_info_path = os.path.join(VCTK_DATA_DIR, "speaker-info.txt")
audio_directory = os.path.join(VCTK_DATA_DIR, "wav48_silence_trimmed")

vctk_dataset = VCTKDataset(speaker_info_path, audio_directory)
vctk_dataloader =DataLoader(vctk_dataset, batch_size=8, shuffle=True, collate_fn=vc_pad_collate)

In [11]:
padded_waveforms, speaker, len_waveforms = next(iter(vctk_dataloader))

In [12]:
padded_waveforms.shape, speaker.shape, len_waveforms.shape

(torch.Size([8, 1, 248242]), torch.Size([8]), torch.Size([8]))

In [14]:
torch.max(len_waveforms)

tensor(248242)